In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Stage 2: Building MVP: - 03 RAG


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/workshops/rag-ops/2.3_mvp_rag.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fworkshops%2Frag-ops%2F2.3_mvp_rag.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/workshops/rag-ops/2.3_mvp_rag.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/workshops/rag-ops/2.3_mvp_rag.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview

This notebook is the third in a series designed to guide you through building a Minimum Viable Product (MVP) for a Multimodal Retrieval Augmented Generation (RAG) system using the Vertex Gemini API.

Building on the foundation of text chunking and embedding generation from the previous notebooks, we now bring those pieces together to construct the core retrieval and generation components of our RAG system. This notebook demonstrates a practical implementation of these components using a ground truth dataset, showcasing how to effectively retrieve relevant information and generate accurate answers.

**Here's what you'll achieve:**

* **Build a Retrieval System:**  Develop a robust retrieval system that leverages semantic search and Vertex AI Embeddings to efficiently identify the most pertinent text chunks for any given query. This system will utilize the vector database created in the previous notebook to perform accurate similarity searches.
* **Implement Generation:**  Construct a generation component that utilizes the retrieved information to produce comprehensive and informative answers. This process involves selecting the top N results from the retrieved chunks and feeding them to Gemini 1.5 Pro and Flash models for generating the final answer.
* **Generate Citations:**  Incorporate functionality to generate citations for the answers produced by your RAG system. This ensures transparency and traceability of the information provided, enhancing the reliability and trustworthiness of your MVP.
* **Test with Ground Truth Data:**  Apply your complete RAG system to a ground truth dataset, observing its performance on real-world questions and answers. This exercise provides valuable insights into the effectiveness of your retrieval and generation pipeline.

This notebook bridges the gap between individual components and a functional RAG system. By implementing retrieval and generation mechanisms and testing them on ground truth data, you gain a practical understanding of how to build an end-to-end solution for answering questions based on multimodal sources.


## Getting Started

### Install Vertex AI SDK for Python


In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

In [2]:
import sys

if "google.colab" in sys.modules:
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.


In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information, GCS Bucket and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
import os
import sys

from google.cloud import storage
import vertexai

PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"
BUCKET_NAME = "mlops-for-genai"

if PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

if not PROJECT_ID or PROJECT_ID == "[your-project-id]" or PROJECT_ID == "None":
    raise ValueError("Please set your PROJECT_ID")


vertexai.init(project=PROJECT_ID, location=LOCATION)

# Initialize cloud storage
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET_NAME)

In [3]:
# # Variables for data location. Do not change.

PRODUCTION_DATA = "multimodal-finanace-qa/data/unstructured/production/"
PICKLE_FILE_NAME = "index_db.pkl"

### Import libraries


In [4]:
# Library

import pickle

from google.cloud import storage
import numpy as np
import pandas as pd
from rich import print as rich_print
from vertexai.generative_models import GenerativeModel, HarmBlockThreshold, HarmCategory
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel

### Load the Gemini 1.5 models

To learn more about all [Gemini API models on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).

The Gemini model family has several model versions. You will start by using Gemini 1.5 Flash. Gemini 1.5 Flash is a more lightweight, fast, and cost-efficient model. This makes it a great option for prototyping.


In [5]:
MODEL_ID_FLASH = "gemini-1.5-flash-001"  # @param {type:"string"}
MODEL_ID_PRO = "gemini-1.5-pro-001"  # @param {type:"string"}


gemini_15_flash = GenerativeModel(MODEL_ID_FLASH)
gemini_15_pro = GenerativeModel(MODEL_ID_PRO)

In [6]:
# @title Helper Functions


def get_load_dataframes_from_gcs():
    gcs_path = "multimodal-finanace-qa/data/embeddings/" + PICKLE_FILE_NAME
    # print("GCS PAth: ", gcs_path)
    blob = bucket.blob(gcs_path)

    # Download the pickle file from GCS
    blob.download_to_filename(f"{PICKLE_FILE_NAME}")

    # Load the pickle file into a list of dataframes
    with open(f"{PICKLE_FILE_NAME}", "rb") as f:
        dataframes = pickle.load(f)

    # Assign the dataframes to variables
    (
        index_db_final,
        extracted_text_chunk_df,
        video_metadata_chunk_df,
        audio_metadata_chunk_df,
    ) = dataframes

    return (
        index_db_final,
        extracted_text_chunk_df,
        video_metadata_chunk_df,
        audio_metadata_chunk_df,
    )

![](https://storage.googleapis.com/mlops-for-genai/multimodal-finanace-qa/img/rag_eval_flow.png)

In [7]:
# Get the data that has been extracted in the previous step: IndexDB.
# Make sure that you have ran the previous notebook: stage_2_mvp_chunk_embeddings.ipynb


(
    index_db_final,
    extracted_text_chunk_df,
    video_metadata_chunk_df,
    audio_metadata_chunk_df,
) = get_load_dataframes_from_gcs()

In [8]:
index_db_final.head()

In [9]:
index_db_final.shape

In [10]:
%%time
# load training data
training_data = pd.read_csv(
    "gs://mlops-for-genai/multimodal-finanace-qa/data/structured/training_data_subset.csv"
)

In [11]:
training_data.shape

In [ ]:
training_data.head(2)

In [ ]:
training_data["question_type"].value_counts()

In [12]:
index = 8

print("*******The question: *******\n")
rich_print(training_data["question"][index])
print("\n*******The ground-truth answer:*******")
rich_print(training_data["answer"][index])
print("\n*******The question type: *******\n", training_data["question_type"][index])
print(
    "*******The question type description: *******\n",
    training_data["question_type_description"][index],
)
print(
    "*******Text citation: *******\n",
)
rich_print(training_data["text_citation"][index])
print(
    "*******Audio citation: *******\n",
)
rich_print(training_data["audio_citation"][index])
print(
    "*******Video citation: *******\n",
)
rich_print(training_data["video_citation"][index])

## Retrieval

In [13]:
# @title Retrieval Functions

import numpy as np
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel


def get_gemini_response(
    model,
    generation_config=None,
    safety_settings=None,
    uri_path=None,
    mime_type=None,
    prompt=None,
):
    if not generation_config:
        generation_config = {
            "max_output_tokens": 8192,
            "temperature": 1,
            "top_p": 0.95,
        }

    if not safety_settings:
        safety_settings = {
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        }

    responses = model.generate_content(
        prompt,
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )
    final_response = []
    for response in responses:
        try:
            final_response.append(response.text)
        except ValueError:
            # print("Something is blocked...")
            final_response.append("blocked")

    return "".join(final_response)


def get_text_embeddings(
    texts: list[str] = ["banana muffins? ", "banana bread? banana muffins?"],
    task: str = "RETRIEVAL_DOCUMENT",
    model_name: str = "textembedding-gecko@003",
) -> list[list[float]]:
    # print("doing...")
    """Embeds texts with a pre-trained, foundational model."""
    model = TextEmbeddingModel.from_pretrained(model_name)
    inputs = [TextEmbeddingInput(text, task) for text in texts]
    embeddings = model.get_embeddings(inputs)
    return [embedding.values for embedding in embeddings][0]


def get_cosine_score(
    dataframe: pd.DataFrame, column_name: str, input_text_embed: np.ndarray
) -> float:
    """
    Calculates the cosine similarity between the user query embedding and the dataframe embedding for a specific column.

    Args:
        dataframe: The pandas DataFrame containing the data to compare against.
        column_name: The name of the column containing the embeddings to compare with.
        input_text_embed: The NumPy array representing the user query embedding.

    Returns:
        The cosine similarity score (rounded to two decimal places) between the user query embedding and the dataframe embedding.
    """
    if dataframe[column_name]:
        text_cosine_score = round(np.dot(dataframe[column_name], input_text_embed), 2)
        return text_cosine_score
    else:
        return 0

## Generation

In [14]:
# @title Generation Functions


def get_answer(row, vector_db, model, top_n=5):
    query_embedding = get_text_embeddings([row["question"]])
    # Find score
    cosine_scores = vector_db.apply(
        lambda x: get_cosine_score(x, "embeddings", query_embedding),
        axis=1,
    )
    # print(len(cosine_scores))
    # Remove same image comparison score when user image is matched exactly with metadata image
    # cosine_scores = cosine_scores[cosine_scores < 1.00000000]
    # Get top N cosine scores and their indices
    top_n_cosine_scores = cosine_scores.nlargest(top_n).index.tolist()
    top_n_cosine_values = cosine_scores.nlargest(top_n).values.tolist()

    citations = vector_db.iloc[top_n_cosine_scores].copy()
    # citations['score'] = top_n_cosine_scores
    citations.loc[:, "score"] = top_n_cosine_values
    citations = citations[["uid", "type", "content", "score"]]

    # # print(citations)
    # gemini_content = get_gemini_content_list(query, vector_db, top_n_cosine_scores)
    context = "\n".join(citations["content"].tolist())
    prompt = f"""Task: Answer the question based on the provided context.

Guidelines:

1. **Chain of Thought:**  Before generating the final answer, break down the question into smaller steps and reason through them logically.
2. **Conciseness:** If the question asks for specific information, provide only those values.
3. **Critical Thinking:** Analyze the entire context thoroughly and critically evaluate the information before formulating your answer. Do not rush to conclusions.
4. **Structure:** Present your answer in a clear and organized manner. It should include everything that the question is asking.
5. **Answer Only:**  Do not include the chain of thought reasoning in the final answer. The answer should be concise.

Question: {row['question']}

Context: {context}

Answer:
"""
    response = get_gemini_response(model=model, prompt=prompt)
    type_mod = np.unique(
        [each_cit["type"] for each_cit in citations.to_dict("records")]
    )
    return (response, context, type_mod, citations.to_dict("records"), prompt)


def get_gen_answer_citation(
    training_data,
    index,
    ref_data=[
        extracted_text_chunk_df,
        video_metadata_chunk_df,
        audio_metadata_chunk_df,
    ],
    top_n=None,
    gcs_full_path=True,
):
    gen_citation = training_data["citation"][index]
    # gen_citation_uids = [each_el['uid'] for each_el in gen_citation]
    final_citations = []
    for each_cit in gen_citation:
        if each_cit["type"] == "text":
            text_citations = ref_data[0][ref_data[0]["uid"].isin([each_cit["uid"]])]
            final_citations.append(
                {
                    "score": each_cit["score"],
                    "uid": each_cit["uid"],
                    "type": each_cit["type"],
                    "content": text_citations["text"].values[0],
                    "gcs_path": (
                        text_citations["gcs_path"].values[0]
                        if gcs_full_path
                        else text_citations["gcs_path"].values[0].split("/")[-1]
                    ),
                }
            )
        elif each_cit["type"] == "video":
            video_citations = ref_data[1][ref_data[1]["uid"].isin([each_cit["uid"]])]
            final_citations.append(
                {
                    "score": each_cit["score"],
                    "uid": each_cit["uid"],
                    "type": each_cit["type"],
                    "content": video_citations["video_description"].values[0],
                    "gcs_path": (
                        video_citations["video_gcs"].values[0]
                        if gcs_full_path
                        else video_citations["video_gcs"].values[0].split("/")[-1]
                    ),
                }
            )

        elif each_cit["type"] == "audio":
            audio_citations = ref_data[2][ref_data[2]["uid"].isin([each_cit["uid"]])]
            final_citations.append(
                {
                    "score": each_cit["score"],
                    "uid": each_cit["uid"],
                    "type": each_cit["type"],
                    "content": audio_citations["audio_description"].values[0],
                    "gcs_path": (
                        audio_citations["audio_gcs"].values[0]
                        if gcs_full_path
                        else audio_citations["audio_gcs"].values[0].split("/")[-1]
                    ),
                }
            )
    if top_n:
        return pd.DataFrame(final_citations[:top_n])
    else:
        return pd.DataFrame(final_citations)

## Answer

if you are getting: InternalServerError: 500 Internal error encountered. Try switching between 001 or 002 models. There might be a capacity issue. You can also re-run the cell after few minutes.

`gemini-1.5-flash-001` or `gemini-1.5-flash-002`

`gemini-1.5-pro-001` or `gemini-1.5-pro-002`

In [15]:
%%time
training_data_flash = training_data.copy()
training_data_flash[["gen_answer", "context", "type_mode", "citation", "prompt"]] = (
    training_data.apply(
        lambda x: get_answer(x, index_db_final, gemini_15_flash, top_n=100),
        axis=1,
        result_type="expand",
    )
)

In [16]:
%%time
training_data_pro = training_data.copy()
training_data_pro[["gen_answer", "context", "type_mode", "citation", "prompt"]] = (
    training_data.apply(
        lambda x: get_answer(x, index_db_final, gemini_15_pro, top_n=100),
        axis=1,
        result_type="expand",
    )
)

### Answer with Gemini 1.5 Flash

In [17]:
training_data_flash[["question", "answer", "gen_answer"]].head()

In [18]:
index = 4

print("*******The question: *******\n")
rich_print(training_data_flash["question"][index])

print("\n*******The ground-truth answer:*******\n")
rich_print(training_data_flash["answer"][index])

print("\n*******The generated answer: *******\n")
rich_print(training_data_flash["gen_answer"][index])

In [19]:
get_gen_answer_citation(training_data_flash, index, top_n=5, gcs_full_path=False)

### Answer with Gemini 1.5 Pro

In [20]:
training_data_pro[["question", "answer", "gen_answer"]].head()

In [21]:
print("*******The question: *******\n")
rich_print(training_data_pro["question"][index])

print("\n*******The ground-truth answer:*******\n")
rich_print(training_data_pro["answer"][index])

print("\n*******The generated answer: *******\n")
rich_print(training_data_pro["gen_answer"][index])

In [22]:
get_gen_answer_citation(training_data_pro, index, top_n=5, gcs_full_path=False)

In [23]:
training_data_pro.shape

### Save the intermediate Files

In [24]:
# # [Optional]
# import pickle

# pickle_file_name ="training_data_results.pkl"
# data_to_dump = [training_data_flash, training_data_pro]

# gcs_location = f"gs://mlops-for-genai/multimodal-finanace-qa/data/structured/{pickle_file_name}"

# with open(f"{pickle_file_name}", "wb") as f:
#     pickle.dump(data_to_dump, f)


# # Upload the pickle file to GCS
# !gsutil cp {pickle_file_name} {gcs_location}